In [1]:
import pandas as pd
import numpy as np

def generate_mardia_latex(dfs, root=''):
    for condition, df in dfs.items():
        # Drop m = 10 due to limitations of pengouin's multivariate_normality function
        df = df[df["m"] != 10]
        
        # Group by Estimator
        groups = df.groupby('Estimator')
                
        # Compute the first and fifth percentile and median
        summary = groups['p-values'].agg([
            ("Minimum", 'min'),
            ('Quantile 1%', lambda x: np.quantile(x, 0.01)),
            ('Quantile 5%', lambda x: np.quantile(x, 0.05)),
            ('Median', 'median'),
        ]).reset_index()

        # Save to LaTeX
        summary.to_latex(
            buf=root + f"{condition}.tex",
            index=False,
            float_format="%.4f",
            caption=f"P-values of Mardia's tests for model residuals: \\textbf{{{condition.upper()}}}",
            label=f"tab:mardia_{condition.replace(' ', '_').lower()}",
            column_format="lrrrr",
            header=["Model", "Minimum", "Quantile 1\\%", "Quantile 5\\%", "Median"],
            escape=False,
            position="H"  # <--- This forces the [H] in the generated .tex file
        )

In [2]:
dfs = {
    "VAR flawed": pd.read_csv('../results/experiment_results_VAR_flawed.csv'), 
    "VAR": pd.read_csv('../results/experiment_results_VAR.csv'), 
    "MAR": pd.read_csv('../results/experiment_results_MAR.csv'),
    "MAR symmetric": pd.read_csv('../results/experiment_results_MAR_symmetric.csv')
}

# Execute the generation
generate_mardia_latex(dfs)